In [22]:
import datetime

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import math

In [14]:
ds = pd.read_excel('EuropeTop100Attractions_ENG_20190101_20210821.xlsx', sheet_name='Attractions')
ds.describe(include='all', datetime_is_numeric=True).T

,count,unique,top,freq
ID,100,100,MAG001,1
Name,100,99,Old Town,2
Country,100,26,Spain,17
ISO,100,25,ES,17


We drop MAG083 which is not in europe, see the main notebook for more explanation.

In [15]:
ds['ID'] = np.where(ds['ID'] == 'MAG083', np.nan, ds['ID'])
ds.dropna(subset=['ID'], inplace=True)

We need to change the name of some attractions because they are not complete or doesn't appear on OpenStreetMap. In order to do this we have done some iteartion looking at address in oder to see if is the rigth one or not and searched the locations on OpenStreetMap.

Some of the new name that we put are not about the attraction, but since we don't need something really precise in term of geolocalization is fine.

In this case we prefered to it manually since is a small number of places, but in case is needed to perform a similar task is better to use the GoogleMaps map which is the same map used by tripAdvisor with a check based on the ISO code. Basically using the ISO code given in the dataset and the one that we can retrieve by the address we can see at least if the attraction is in the right country.

In [16]:
ds['Name'] = np.where(ds['Name'] == 'The Alhambra', 'Alhambra', ds['Name'])
ds['Name'] = np.where(ds['Name'] == "Krakow's Rynek Glowny Central Square", "Krakow's Rynek Glowny", ds['Name'])
ds['Name'] = np.where(ds['Name'] == "Pompeii Archaeological Park", 'Pompeii', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Historic Center of Vienna', 'Vienna', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Palace of Catalan Music', 'Palau de la Música Catalana', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Old Town', 'Dubrovnik', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Park and National Palace of Pena', 'National Palace of Pena', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Old Town', 'Old Town Warsaw', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Historic Areas of Istanbul', 'Istanbul', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'The State Museum St. Isaac’s Cathedral', ' St. Isaac’s Cathedral', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Ancient City of Ephesus', 'Ephesus', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Catedral Primada', 'Toledo Cathedral', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Palazzo di Diocleziano', "Diocletian's Palace", ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'I Trulli di Alberobello - World Heritage Site', 'Alberobello', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Basilica Papale e Sacro Convento...', 'Piazza della Porziuncola', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Sea Fortress Suomenlinna', 'Suomenlinna', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Parco Acheologico di Ercolano', 'Ercolano', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Historic Centre of San Gimignano', 'San Gimignano', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Alcázar of Segovia', 'Segovia', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Kotor Old City', 'Kotor', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Bom Jesus do Monte', 'Funicular do Bom Jesus do Monte ', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'The Walls of Avila', 'Ávila Walls', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Centro Storico', 'Centro Storico Florence', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Basilica of the Sagrada Familia', 'Sagrada Familia', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Schönbrunn Gardens', 'Schonbrunn Palace', ds['Name'])
ds['Name'] = np.where(ds['Name'] == 'Acueduct of Segovia', '	Aqueduct of Segovia', ds['Name'])

## Adding attraction coordinates
We used geopy and Nominatim library to put coordinates to attractions. <br>
Some of the coordinates were not identified correctly, we checked them manually with the help of OpenStreetMaps, service which is as well based on Nominatim database. After several iterations we were able to match every location with the correct adress. <br>
While working on adresses we found out that attraction MAG083 doesn't belong to Europe, it's located in Curacao, further we excluded it from the dataset
<br>


In [17]:
geolocator = Nominatim(user_agent="coordinates")

for i in ds.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(ds['Name'][i])
        
        #append lat/long to column using dataframe location
        ds.loc[i,'location_lat'] = location.latitude
        ds.loc[i,'location_long'] = location.longitude
        ds.loc[i,'location_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        ds.loc[i,'location_lat'] = ""
        ds.loc[i,'location_long'] = ""
        ds.loc[i,'location_address'] = ""

#print first rows as sample
ds.head(100)

,ID,Name,Country,ISO,location_lat,location_long,location_address
0,MAG001,Sagrada Familia,Spain,ES,41.403479,2.174410,"Basílica de la Sagrada Família, 401, Carrer de..."
1,MAG002,Parc Guell,Spain,ES,41.414235,2.152458,"Park Güell, Carrer d'Olot, la Salut, Gràcia, B..."
2,MAG003,Tower of London,England,UK,51.508120,-0.076222,"Tower of London, Tower Hill, Tower Liberty, Wh..."
3,MAG004,Casa Batllo,Spain,ES,41.391545,2.164696,"Casa Batlló, 43, Passeig de Gràcia, la Dreta d..."
4,MAG005,Staromestske namesti,Czech Republic,CZ,50.087455,14.420967,"Staroměstské náměstí, Staré Město, Praha 1, Pr..."
...,...,...,...,...,...,...,...
95,MAG096,Duomo di Pisa,Italy,IT,43.723283,10.395728,"Cattedrale di Santa Maria Assunta, Piazza del ..."
96,MAG097,La Lonja de la Seda,Spain,ES,39.474237,-0.378562,"La Llotja de la Seda, Plaça del Mercat, el Mer..."
97,MAG098,Ávila Walls,Spain,ES,40.656417,-4.707537,"Murallas de Ávila, Bajada San Nicolás, Ávila, ..."
98,MAG099,Place Stanislas,France,FR,48.693524,6.183286,"Place Stanislas, Saint-Nicolas - Charles III -..."


We have to change scale to latitude and longitude because otherwise will broke the algorithm below.

In [19]:
# We multiply our number per 10000 in order to use int instead of float, because otherwise the function validate dosen't work
points = ds[['location_lat','location_long']]
array = np.full((99, 2), 10000, dtype=int)
points = np.multiply(points,array)
points = points.astype(int) 
print(points)

    location_lat  location_long
0         414034          21744
1         414142          21524
2         515081           -762
3         413915          21646
4         500874         144209
..           ...            ...
95        437232         103957
96        394742          -3785
97        406564         -47075
98        486935          61832
99        437692         112548

[99 rows x 2 columns]


In [20]:
from typing import List, Tuple

def create_clusters(
    number_of_clusters: int, points: List[Tuple[float, float]]
) -> np.ndarray:
    kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(points)
    l_array = np.array([[label] for label in kmeans.labels_])
    clusters = np.append(points, l_array, axis=1)
    return clusters

We used k-means becuase euclidian distance is fine for our case, we only have a couple of places that are in the wrong cluster but overall is a good result. Also hirarchical clustering can be used, but with k-means is easy to define the number of clusters since we have 25 countries and 100 attractions we started with a k in middle between them, see the result and change it until we reached a good result.

In [23]:
ds['cluster'] = create_clusters(40,points)[:,2]
ds.sort_values(by='cluster').head(30)

,ID,Name,Country,ISO,location_lat,location_long,location_address,cluster
0,MAG001,Sagrada Familia,Spain,ES,41.403479,2.174410,"Basílica de la Sagrada Família, 401, Carrer de...",0
1,MAG002,Parc Guell,Spain,ES,41.414235,2.152458,"Park Güell, Carrer d'Olot, la Salut, Gràcia, B...",0
22,MAG023,Casa Milà - La Pedrera,Spain,ES,41.395400,2.161762,"Casa Milà, 261-265, Carrer de Provença, la Dre...",0
3,MAG004,Casa Batllo,Spain,ES,41.391545,2.164696,"Casa Batlló, 43, Passeig de Gràcia, la Dreta d...",0
56,MAG057,Historic Fortified City of Carcassonne,France,FR,43.206453,2.363922,"Ville fortifiée historique de Carcassonne, Car...",0
29,MAG030,Palau de la Música Catalana,Spain,ES,41.387584,2.175232,"Palau de la Música Catalana, 4-6, Carrer del P...",0
10,MAG011,Acropolis,Greece,GR,37.971689,23.726320,"Ακρόπολη, Περίπατος Ακρόπολης (Βόρεια κλιτύς),...",1
19,MAG020,Seine River,France,FR,49.682532,-96.716103,"Seine River, Division No. 2, Manitoba, Canada",2
41,MAG042,Danube River,Hungary,HU,-20.265066,57.380554,"Danube Street, Jardin d'Anna, Black River, Out...",3
87,MAG088,Vilnius Old Town,Lithuania,LT,54.681873,25.288404,"Senamiestis, Senamiesčio seniūnija, Vilnius, V...",4


We save the result for merging in to the other data set in the main file.

In [24]:
ds.to_excel('GeocodingAttractions_group40.xlsx', index=False, sheet_name='Attractions')
print(datetime.datetime.now())

2023-01-11 11:28:19.202245
